In [344]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from copy import deepcopy
import random
import tkinter as tk

In [345]:
square_size = 25
chessboard_shape = (9,6)
images_temp = []
manual_corners = []

train_images_path = 'train_images2'
test_images_path = 'test_images/1.jpg'
plots_path = 'plots'
video_path = 'video/chessboard.mp4'
parameters_path = 'calibrations'

VIDEO = False

In [346]:
def load_camera_parameters(dir_path):
    return list(sorted([
        np.load(os.path.join(dir_path, fn))
        for fn in os.listdir(dir_path)
        if fn.endswith(".npz")
    ], key=lambda x: x["run"]))


In [347]:
def draw_cube(current_image, cam_matrix, r_vec, t_vec, color, side_length):

    cube = np.float32([[0, 0, 0], [0, 1, 0], [1, 1, 0], [1, 0, 0],
                       [0, 0, -1], [0, 1, -1], [1, 1, -1], [1, 0, -1]]) * side_length

    cube_pixels, _ = cv2.projectPoints(cube.astype(np.float32), r_vec, t_vec, cam_matrix, None)
    cube_pixels = cube_pixels.astype(np.int32).reshape(-1, 2)

    cv2.drawContours(current_image, [cube_pixels[0:4]], -1, color, 2, lineType=cv2.LINE_AA)

    for i, j in zip(range(4), range(4, 8)):
        cv2.line(current_image, tuple(cube_pixels[i]), tuple(cube_pixels[j]), color, 2, lineType=cv2.LINE_AA)

    cv2.drawContours(current_image, [cube_pixels[4:]], -1, color, 2, lineType=cv2.LINE_AA)


In [348]:
def draw_frame_axis(current_image, cam_matrix, r_vec, t_vec, origin, axis_length):
    axis = np.float32([[1, 0, 0], [0, 1, 0], [0, 0, -1]]) * axis_length
    
    print(r_vec)
    print(t_vec)

    axis_pixels, _ = cv2.projectPoints(axis.astype(np.float32), r_vec, t_vec, cam_matrix, None)
    axis_pixels = axis_pixels.astype(np.int32)

    origin = origin.astype(np.int32)
    cv2.arrowedLine(current_image, origin, axis_pixels[0][0], (255, 0, 0), 3, line_type=cv2.LINE_AA) # Red - x
    cv2.arrowedLine(current_image, origin, axis_pixels[1][0], (0, 255, 0), 3, line_type=cv2.LINE_AA) # Green - y
    cv2.arrowedLine(current_image, origin, axis_pixels[2][0], (0, 0, 255), 3, line_type=cv2.LINE_AA) # Blue - z

In [349]:
def process_frame(frame, corners, calibration):
    global default_object_points
    print("MTX", calibration["mtx"])

    ret, r_vec, t_vec = cv2.solvePnP(default_object_points, corners, calibration["mtx"], None)

    cv2.drawChessboardCorners(frame, chessboard_shape, corners, ret)
    draw_frame_axis(frame, calibration["mtx"], r_vec, t_vec, corners[0][0], 75)
    draw_cube(frame, calibration["mtx"], r_vec, t_vec, (255, 255, 0), 50)

In [350]:
def manual_corner_selection(event, x, y, flags, param):
    global manual_corners, images_temp

    # Left click to add a corner if not all 4 placed
    if event == cv2.EVENT_LBUTTONDOWN and len(manual_corners) < 4:
        # Add corner to list of corners
        manual_corners.append([x, y])

        # Copy last frame to make changes to it
        current_image = deepcopy(images_temp[-1])

        # Add text near click
        cv2.putText(current_image, str(len(manual_corners)), manual_corners[-1],
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1,
                    color=(0, 0, 255), thickness=2, lineType=cv2.LINE_AA)

        # Draw circle at click
        cv2.circle(current_image, manual_corners[-1],
                   radius=3, color=(0, 255, 0), thickness=-1)

        # Connect corners with line
        if len(manual_corners) > 1:
            cv2.line(current_image, manual_corners[-2], manual_corners[-1],
                     color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)

        if len(manual_corners) == 4:
            cv2.line(current_image, manual_corners[0], manual_corners[-1],
                     color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)

        # Save new frame
        images_temp.append(current_image)
        cv2.imshow("Manual Corners Selection (Press any key when all corners selected)", current_image)
    # Right click to remove a corner if one already is placed
    elif event == cv2.EVENT_RBUTTONDOWN and len(manual_corners) > 0:
        # Removed last placed corner and return to previous frame
        manual_corners.pop()
        images_temp.pop()
        current_image = images_temp[-1]
        cv2.imshow("Manual Corners Selection (Press any key when all corners selected)", current_image)

In [351]:
def interpolate_points_from_manual_corners(corners, chessboard_shape, use_perspective_transform=True):
    if len(corners) != 4:
        show_warning("incorrect_num_corners")
        return None

    # Sort corners to (top-left, top-right, bottom-right, bottom-left)
    corners = sort_corners_clockwise(corners, origin="top-left")

    # Calculate the maximum width and height
    max_width = max(np.linalg.norm(corners[1] - corners[0]), np.linalg.norm(corners[3] - corners[2]))
    max_height = max(np.linalg.norm(corners[2] - corners[1]), np.linalg.norm(corners[3] - corners[0]))

    # Horizontal and vertical step calculation using chessboard shape
    horizontal_step = max_width / (chessboard_shape[1] - 1)
    vertical_step = max_height / (chessboard_shape[0] - 1)

    interpolated_row = []
    interpolated_points = []
    # Perform perspective transform for accuracy improvement
    if use_perspective_transform:
        # Use maximum width and height to form destination coordinates for perspective transform
        dest_corners = np.float32([[0, 0], [max_width - 1, 0],
                                   [max_width - 1, max_height - 1], [0, max_height - 1]])
        p_matrix = cv2.getPerspectiveTransform(corners, dest_corners)

        # Get inverse matrix for projecting points from the transformed space back to the original image space
        inverted_p_matrix = np.linalg.inv(p_matrix)

        # Compute each projected point
        for y in range(0, chessboard_shape[0]):
            for x in range(0, chessboard_shape[1]):
                # Calculate the position of the current point relative to the grid using homogenous coordinates
                point = np.array([x * horizontal_step, y * vertical_step, 1])

                # Multiply with inverse matrix to project point from transformed space back to original image space
                point = np.matmul(inverted_p_matrix, point)

                # Divide point by its Z
                point /= point[2]

                # Append the X and Y of point to the list of interpolated points in row
                interpolated_row.append(point[:2])
            # Append interpolated points in row to interpolated points
            interpolated_points.append(interpolated_row)
            interpolated_row = []
    # Flat interpolation
    else:
        for y in range(0, chessboard_shape[0]):
            for x in range(0, chessboard_shape[1]):
                # Calculate the position of the current point relative to the grid
                point = np.array([x * horizontal_step, y * vertical_step])

                # Interpolate the position of the current point between the known corners
                point = corners[0] + point

                # Append the point to the list of interpolated points in row
                interpolated_row.append(point)
            # Append interpolated points in row to interpolated points
            interpolated_points.append(interpolated_row)
            interpolated_row = []

    # If change_point_order is True then point order will start from bottom-left and end on top-right
    # moving through rows before changing column
    # if False then point order will start at top-left and end on bottom-right
    # moving through columns before changing row as already saved
    interpolated_points = np.array(interpolated_points, dtype="float32")
    if chessboard_shape[0] > chessboard_shape[1]:
        interpolated_points = np.flip(interpolated_points, axis=0)
        interpolated_points = np.transpose(interpolated_points, (1, 0, 2))

    # Return (MxN, 1, 2) array to match automatic corner detection output
    return np.reshape(interpolated_points, (-1, 1, 2))

In [352]:
def sort_corners_clockwise(corners, origin="top-left"):
    """
    Sorts given corner coordinates in clockwise order.

    :param corners: array of corner points ([x, y])
    :param origin: which corner point starts the clockwise order (bottom-left, top-left, top-right, or bottom-right)
    :return: returns array of sorted corners
    """
    # Calculate the centroid of the corners
    centroid = np.mean(corners, axis=0)

    # Sort corners and determine their relative position to the centroid
    top = sorted([corner for corner in corners if corner[1] < centroid[1]], key=lambda point: point[0])
    bottom = sorted([corner for corner in corners if corner[1] >= centroid[1]], key=lambda point: point[0],
                    reverse=True)

    # Sort top and bottom corners depending on first element
    if origin == "top-left":
        return np.array(top + bottom, dtype="float32")
    elif origin == "top-right":
        return np.array([top[1]] + bottom + [top[0]], dtype="float32")
    elif origin == "bottom-right":
        return np.array(bottom + top, dtype="float32")
    else:
        return np.array([bottom[1]] + top + [bottom[0]], dtype="float32")

In [353]:

# Loading calibrations parameters
calibrations = load_camera_parameters(parameters_path)
print("Calibrations", calibrations)

# Defining object points
default_object_points = np.zeros((chessboard_shape[0] * chessboard_shape[1], 3), dtype=np.float32)
default_object_points[:, :2] = np.mgrid[0:chessboard_shape[0], 0:chessboard_shape[1]].T.reshape(-1,2) * square_size

# If a test image path is specified then use it
if VIDEO == False:
    print("Read the test image")
    test_image = cv2.imread(test_images_path)
    
    print("Resize the image")
    test_image = cv2.resize(test_image, (0, 0), fx=0.5, fy=0.5)
    
    print("Turn it to greyscale")
    test_gray = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    print("Try to find the corners automatically")
    ret, corners = cv2.findChessboardCorners(test_gray, chessboard_shape,
                                                  flags=cv2.CALIB_CB_ADAPTIVE_THRESH +
                                                        cv2.CALIB_CB_NORMALIZE_IMAGE +
                                                        cv2.CALIB_CB_FAST_CHECK +
                                                        cv2.CALIB_CB_FILTER_QUADS)
    
    
    
    if not ret:
        # show_warning("no_automatic_corners")

        # Corner selection window, calls callback function
        images_temp.append(deepcopy(test_image))
        cv2.imshow("Manual Corners Selection (Press any key when all corners selected)", test_image)
        cv2.setMouseCallback("Manual Corners Selection (Press any key when all corners selected)",
                                 manual_corner_selection)
        # Loop until 4 corners selected and any key is pressed
        while True:
            cv2.waitKey(0)
            if len(manual_corners) == 4:
                cv2.destroyAllWindows()
                break

        # Corner interpolation using selected corners
        corners = interpolate_points_from_manual_corners(manual_corners, chessboard_shape)

        # Reset parameters used for manual corner detection and go back to original image
        current_image = images_temp[0]
        images_temp = []
        manual_corners = []
    
    
    
    

    # Otherwise open a window
    cv2.namedWindow("Test Image")
    cv2.setWindowTitle("Test Image", "Calibration N.1")

    calibration_index = 0
    
    print(calibrations[calibration_index])

    current_image = deepcopy(test_image)
    process_frame(current_image, corners, calibrations[calibration_index])

    cv2.imshow("Test Image", current_image)

    key = cv2.waitKey(0)

    cv2.destroyAllWindows()

# elif VIDEO == True:
#     src_video_flow = cv2.VideoCapture(0)

#     # If it's not open then trying to open it manually may work
#     if not src_video_flow.isOpened():
#         src_video_flow.open()

#     # If the video flow is still closed then raise an exception
#     if not src_video_flow.isOpened():
#         raise Exception("[CAM_STREAM]: Cannot open the camera stream!")

#     calibration_index = 0

#     cv2.namedWindow("RealTime Webcam Video")
#     cv2.setWindowTitle("RealTime Webcam Video", "Calibration N.1")

#     while True:
#         ret, current_frame = src_video_flow.read()

#         if not ret:
#             raise Exception("[CAM_RECV_NEXT_FRAME]: Cannot read the next video frame!")

#         key = cv2.waitKey(1)

#         # If the user presses d then it shows the result with the calibration parameters estimated in the next run
#         if key == ord('d'):
#             calibration_index = (calibration_index + 1) % len(calibrations)
#             cv2.setWindowTitle("RealTime Webcam Video", "Calibration N." + str(calibration_index + 1))
#         # If the user presses a then it shows the result with the calibration parameters estimated in the
#         # previous run
#         elif key == ord('a'):
#             calibration_index = (calibration_index - 1) % len(calibrations)
#             cv2.setWindowTitle("RealTime Webcam Video", "Calibration N." + str(calibration_index + 1))
#         # If the user presses q then the script ends
#         elif key == ord('q'):
#             break

#         # Compute a grayscale version of the current frame and try to detect the chessboard corners automatically
#         frame_gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)

#         found, coords = cv2.findChessboardCorners(frame_gray, chessboard_shape,
#                                                     flags=cv2.CALIB_CB_ADAPTIVE_THRESH +
#                                                         cv2.CALIB_CB_NORMALIZE_IMAGE +
#                                                         cv2.CALIB_CB_FAST_CHECK +
#                                                         cv2.CALIB_CB_FILTER_QUADS)

#         if found:
#             process_frame(current_frame, coords, calibrations[calibration_index], chessboard_shape)

#         cv2.imshow("RealTime Webcam Video", current_frame)

    cv2.destroyAllWindows()
    # src_video_flow.release()


Calibrations [<numpy.lib.npyio.NpzFile object at 0x000001CF2330DAD0>, <numpy.lib.npyio.NpzFile object at 0x000001CF23691450>, <numpy.lib.npyio.NpzFile object at 0x000001CF237AEA10>, <numpy.lib.npyio.NpzFile object at 0x000001CF237AEA50>]
Read the test image
Resize the image
Turn it to greyscale
Try to find the corners automatically


KeyboardInterrupt: 